In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
# 1. 데이터 로드
URL = "/content/drive/MyDrive/SIL/WORK/DSS 머신러닝2/" # 데이터 위치 설정
df = pd.read_csv(URL+"data/NULL_KNN.csv")

In [ ]:
df.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '581', '582', '583', '584', '585', '586', '587', '588', '589',
       'Pass/Fail'],
      dtype='object', length=591)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 591 entries, 0 to Pass/Fail
dtypes: float64(590), int64(1)
memory usage: 7.1 MB


In [ ]:
df.isnull().sum().sum()

41951

In [ ]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1
5,2946.25,2432.84,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5287,0.0167,...,44.0077,0.4949,0.0189,0.0044,3.8276,0.0342,0.0151,0.0052,44.0077,-1
6,3030.27,2430.12,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.5816,-0.0270,...,NaN,0.5010,0.0143,0.0042,2.8515,0.0342,0.0151,0.0052,44.0077,-1
7,3058.88,2690.15,2248.9000,1004.4692,0.7884,100.0,106.2400,0.1185,1.5153,0.0157,...,95.0310,0.4984,0.0106,0.0034,2.1261,0.0204,0.0194,0.0063,95.0310,-1
8,2967.68,2600.47,2248.9000,1004.4692,0.7884,100.0,106.2400,0.1185,1.5358,0.0111,...,111.6525,0.4993,0.0172,0.0046,3.4456,0.0111,0.0124,0.0045,111.6525,-1
9,3016.11,2428.37,2248.9000,1004.4692,0.7884,100.0,106.2400,0.1185,1.5381,0.0159,...,90.2294,0.4967,0.0152,0.0038,3.0687,0.0212,0.0191,0.0073,90.2294,-1


In [ ]:
print("불량 유형 확인")
print(df['Pass/Fail'].unique())
print(" ")
print("불량 유형별 수 확인")
print(df['Pass/Fail'].value_counts())

불량 유형 확인
[-1  1]
 
불량 유형별 수 확인
-1    1463
 1     104
Name: Pass/Fail, dtype: int64


In [ ]:
data_X = df.drop(["Pass/Fail"], axis = 1)
data_Y = df['Pass/Fail']

In [ ]:
data_X.describe()

,0,1,2,3,4,5,6,7,8,9,...,580,581,582,583,584,585,586,587,588,589
count,1561.000000,1560.000000,1553.000000,1553.000000,1553.000000,1553.0,1553.000000,1558.000000,1565.000000,1565.000000,...,618.000000,618.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000
mean,3014.452896,2495.850231,2200.547318,1396.376627,4.197013,100.0,101.112908,0.121822,1.462862,-0.000841,...,0.005396,97.934373,0.500096,0.015318,0.003847,3.067826,0.021458,0.016475,0.005283,99.670066
std,73.621787,80.407705,29.513152,441.691640,56.355540,0.0,6.237214,0.008961,0.073897,0.015116,...,0.003116,87.520966,0.003404,0.017180,0.003720,3.578033,0.012358,0.008808,0.002867,93.891919
min,2743.240000,2158.750000,2060.660000,0.000000,0.681500,100.0,82.131100,0.000000,1.191000,-0.053400,...,0.001000,0.000000,0.477800,0.006000,0.001700,1.197500,-0.016900,0.003200,0.001000,0.000000
25%,2966.260000,2452.247500,2181.044400,1081.875800,1.017700,100.0,97.920000,0.121100,1.411200,-0.010800,...,0.003400,46.184900,0.497900,0.011600,0.003100,2.306500,0.013425,0.010600,0.003300,44.368600
50%,3011.490000,2499.405000,2201.066700,1285.214400,1.316800,100.0,101.512200,0.122400,1.461600,-0.001300,...,0.004700,72.288900,0.500200,0.013800,0.003600,2.757650,0.020500,0.014800,0.004600,71.900500
75%,3056.650000,2538.822500,2218.055500,1591.223500,1.525700,100.0,104.586700,0.123800,1.516900,0.008400,...,0.006475,116.539150,0.502375,0.016500,0.004100,3.295175,0.027600,0.020300,0.006400,114.749700
max,3356.350000,2846.440000,2315.266700,3715.041700,1114.536600,100.0,129.252200,0.128600,1.656400,0.074900,...,0.028600,737.304800,0.509800,0.476600,0.104500,99.303200,0.102800,0.079900,0.028600,737.304800


In [ ]:
def print_all_reg(Y_test,pred):
    # Specificity를 구하기 위해 confusion matrix를 이용

    cm1 = confusion_matrix(Y_test,pred)
    specificity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])

    #결과 검사
    #recall = cm1[1,1]/(cm1[1,1]+cm1[1,0])
    #pre = cm1[1,1]/(cm1[1,1]+cm1[0,1])

    G_mean = recall_score(Y_test,pred,average = 'weighted') * specificity1
    print("model의 recall 값은 {:.3f}".format(recall_score(Y_test,pred,average = 'weighted')))
    print("model의 2종 오류 확률 값은 {:.3f}".format(1-recall_score(Y_test,pred,average = 'weighted')))
    print("model의 Specificity 값은 {:.3f}".format(specificity1))
    print("model의 1종 오류 확률 값은 {:.3f}".format(1-specificity1))
    print("model의 precision 값은 {:.3f}".format(precision_score(Y_test,pred,average = 'weighted')))
    print("model의 f1_score 값은 {:.3f}".format(f1_score(Y_test,pred,average = 'weighted')))
    print("model의 G-mean 값은 {:.3f}".format(np.sqrt(G_mean)))
    print("model의 accuracy 값은 {:.3f}".format(accuracy_score(Y_test,pred)))

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

#Train test 분리

# 0으로 Imputer 선언
data_X_zero = data_X.fillna(0)
training_data_zero, validation_data_zero, training_labels_zero, validation_labels_zero = train_test_split(data_X_zero, data_Y, test_size = 0.2, random_state = 100)

# 최빈값으로 Imputer 선언
imputer_mean = SimpleImputer(strategy='most_frequent')
data_X_mean = imputer_mean.fit_transform(data_X)
training_data_mean, validation_data_mean, training_labels_mean, validation_labels_mean = train_test_split(data_X_mean, data_Y, test_size = 0.2, random_state = 100)

# KNN으로 Imputer 선언
imputer_knn = KNNImputer(n_neighbors=5)
data_X_knn = imputer_knn.fit_transform(data_X)
training_data_knn, validation_data_knn, training_labels_knn, validation_labels_knn = train_test_split(data_X_knn, data_Y, test_size = 0.2, random_state = 100)


In [ ]:
classifier = LogisticRegression(max_iter = 1000)
classifier2 = LogisticRegression(max_iter = 1000)
classifier3 = LogisticRegression(max_iter = 1000)


# 0으로 대치
classifier.fit(training_data_zero,training_labels_zero)
train_preds = classifier.predict(training_data_zero)
preds_zero = classifier.predict(validation_data_zero)
print(classifier.score(validation_data_zero, validation_labels_zero))
print_all_reg(validation_labels_zero,preds_zero)
print(" ")

# 평균
classifier2.fit(training_data_mean,training_labels_mean)
train_preds_mean = classifier2.predict(training_data_mean)
preds_mean = classifier2.predict(validation_data_mean)
print(classifier2.score(validation_data_mean, validation_labels_mean))
print_all_reg(validation_labels_mean,preds_mean)
print(" ")

# KNN
classifier3.fit(training_data_knn,training_labels_knn)
train_preds_knn = classifier3.predict(training_data_knn)
preds_knn = classifier3.predict(validation_data_knn)
print(classifier3.score(validation_data_knn, validation_labels_knn))
print_all_reg(validation_labels_knn,preds_knn)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9140127388535032
model의 recall 값은 0.914
model의 2종 오류 확률 값은 0.086
model의 Specificity 값은 0.980
model의 1종 오류 확률 값은 0.020
model의 precision 값은 0.869
model의 f1_score 값은 0.891
model의 G-mean 값은 0.946
model의 accuracy 값은 0.914
 


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9140127388535032
model의 recall 값은 0.914
model의 2종 오류 확률 값은 0.086
model의 Specificity 값은 0.980
model의 1종 오류 확률 값은 0.020
model의 precision 값은 0.869
model의 f1_score 값은 0.891
model의 G-mean 값은 0.946
model의 accuracy 값은 0.914
 
0.910828025477707
model의 recall 값은 0.911
model의 2종 오류 확률 값은 0.089
model의 Specificity 값은 0.976
model의 1종 오류 확률 값은 0.024
model의 precision 값은 0.869
model의 f1_score 값은 0.890
model의 G-mean 값은 0.943
model의 accuracy 값은 0.911


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# 문제 1. Validation data에 fit 사용 (데이터 분석 대회에서는 부정행위) - 테스트 데이터는 처음 주어진다고 가정해야함
# 문제 2. Max_iter = 500 (default) 파라미터 값을 몇번 업데이트 할것인지 - ConvergenceWarning는 스케일링 여부와 연관성이 높음

from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

#Train test 분리
# min max scaling
scaler = MinMaxScaler()


# 0으로 Imputer 선언
data_X_zero = data_X.fillna(0)
training_data_zero, validation_data_zero, training_labels_zero, validation_labels_zero = train_test_split(data_X_zero, data_Y, test_size = 0.2, random_state = 100)

#스케일링
scaler.fit(training_data_zero)
training_data_zero = scaler.transform(training_data_zero)
validation_data_zero = scaler.transform(validation_data_zero)

# 최빈값으로 Imputer 선언 & training_data로 fit하고 validation에 적용
imputer_mean = SimpleImputer(strategy='most_frequent')
training_data_mean, validation_data_mean, training_labels_mean, validation_labels_mean = train_test_split(data_X_mean, data_Y, test_size = 0.2, random_state = 100)
training_data_mean = imputer_mean.fit_transform(training_data_mean)
validation_data_mean = imputer_mean.transform(validation_data_mean)

#스케일링
scaler.fit(training_data_mean)
training_data_mean = scaler.transform(training_data_mean)
validation_data_mean = scaler.transform(validation_data_mean)

# KNN으로 Imputer 선언
imputer_knn = KNNImputer(n_neighbors=5)
training_data_knn, validation_data_knn, training_labels_knn, validation_labels_knn = train_test_split(data_X_knn, data_Y, test_size = 0.2, random_state = 100)
training_data_knn = imputer_mean.fit_transform(training_data_knn)
validation_data_knn = imputer_mean.transform(validation_data_knn)

#스케일링
scaler.fit(training_data_knn)
training_data_knn = scaler.transform(training_data_knn)
validation_data_knn = scaler.transform(validation_data_knn)

In [ ]:
classifier = LogisticRegression(max_iter = 1000)
classifier2 = LogisticRegression(max_iter = 1000)
classifier3 = LogisticRegression(max_iter = 1000)


# 0으로 대치
classifier.fit(training_data_zero,training_labels_zero)
train_preds = classifier.predict(training_data_zero)
preds_zero = classifier.predict(validation_data_zero)
print(classifier.score(validation_data_zero, validation_labels_zero))
print_all_reg(validation_labels_zero,preds_zero)
print(" ")

# 평균
classifier2.fit(training_data_mean,training_labels_mean)
train_preds_mean = classifier2.predict(training_data_mean)
preds_mean = classifier2.predict(validation_data_mean)
print(classifier2.score(validation_data_mean, validation_labels_mean))
print_all_reg(validation_labels_mean,preds_mean)
print(" ")

# KNN
classifier3.fit(training_data_knn,training_labels_knn)
train_preds_knn = classifier3.predict(training_data_knn)
preds_knn = classifier3.predict(validation_data_knn)
print(classifier3.score(validation_data_knn, validation_labels_knn))
print_all_reg(validation_labels_knn,preds_knn)

0.9267515923566879
model의 recall 값은 0.927
model의 2종 오류 확률 값은 0.073
model의 Specificity 값은 0.993
model의 1종 오류 확률 값은 0.007
model의 precision 값은 0.870
model의 f1_score 값은 0.898
model의 G-mean 값은 0.959
model의 accuracy 값은 0.927
 
0.9267515923566879
model의 recall 값은 0.927
model의 2종 오류 확률 값은 0.073
model의 Specificity 값은 0.993
model의 1종 오류 확률 값은 0.007
model의 precision 값은 0.870
model의 f1_score 값은 0.898
model의 G-mean 값은 0.959
model의 accuracy 값은 0.927
 
0.9299363057324841
model의 recall 값은 0.930
model의 2종 오류 확률 값은 0.070
model의 Specificity 값은 0.997
model의 1종 오류 확률 값은 0.003
model의 precision 값은 0.871
model의 f1_score 값은 0.899
model의 G-mean 값은 0.963
model의 accuracy 값은 0.930


In [ ]:
# 문제 1. Validation data에 fit 사용 (데이터 분석 대회에서는 부정행위) - 테스트 데이터는 처음 주어진다고 가정해야함
# 문제 2. Max_iter = 500 (default) 파라미터 값을 몇번 업데이트 할것인지 - ConvergenceWarning는 스케일링 여부와 연관성이 높음

# KNN imputer 최적의 k 찾기
data_X = df.drop(["Pass/Fail"], axis = 1)
data_Y = df['Pass/Fail']

K = list(range(1,10,1)) # 탐색할 K를 리스트에 저장
K_acc = []
for k in K: # K에 포함된 수만큼 반복

  # 학습/검증 데이터 분할 \:코드가 길어져서 아래로 내려서 보기위해 사용
  training_data_knn, validation_data_knn, training_labels_knn, validation_labels_knn = \
  train_test_split(data_X, data_Y, test_size = 0.2, random_state = 100)

  # 결측치 보완
  print(" ")
  imputer_knn = KNNImputer(n_neighbors=k) # 모델 정의
  training_data_knn = imputer_knn.fit_transform(training_data_knn) # 학습 및 변환
  validation_data_knn = imputer_knn.transform(validation_data_knn) # 변환 [문제 1 주의]

  # min max scaling
  scaler = MinMaxScaler()
  scaler.fit(training_data_knn)
  training_data_knn = scaler.transform(training_data_knn)
  validation_data_knn = scaler.transform(validation_data_knn) # 평가 데이터는 transform만 진행해야함

  # 평가 모델 정의
  classifier = LogisticRegression(max_iter = 1000) # 문제 2 주의
  classifier.fit(training_data_knn,training_labels_knn)
  preds_knn = classifier.predict(validation_data_knn)

  # 성능 평가
  test_score = accuracy_score(validation_labels_knn,preds_knn)
  K_acc.append(test_score)

  print("n_neighbors: {}".format(k))
  print("test_score: {}",format(test_score))

print("K_acc: ",np.array(K_acc))
# 최적의 K 찾기
print(" ")
print("최적의 K")
print(K[np.argmax(np.array(K_acc))])
print(" ")
print("최적의 K인 경우 acc:")
print(K_acc[np.argmax(np.array(K_acc))])

# argmax 예시
# np.argmax([1,2,3,4,5]) = 4
# np.argmax([1,4,2]) = 1

 
n_neighbors: 1
test_score: {} 0.9267515923566879
 
n_neighbors: 2
test_score: {} 0.9267515923566879
 
n_neighbors: 3
test_score: {} 0.9267515923566879
 
n_neighbors: 4
test_score: {} 0.9299363057324841
 
n_neighbors: 5
test_score: {} 0.9299363057324841
 
n_neighbors: 6
test_score: {} 0.9267515923566879
 
n_neighbors: 7
test_score: {} 0.9267515923566879
 
n_neighbors: 8
test_score: {} 0.9299363057324841
 
n_neighbors: 9
test_score: {} 0.9299363057324841
K_acc:  [0.92675159 0.92675159 0.92675159 0.92993631 0.92993631 0.92675159
 0.92675159 0.92993631 0.92993631]
 
최적의 K
4
 
최적의 K인 경우 acc:
0.9299363057324841
